In [ ]:
import numpy as np
import math
import sys
from pathlib import Path
from typing import List, Tuple, Optional, Iterable, Dict, Set
from scipy.spatial import cKDTree

In [ ]:
# ---------------------------- OBJ parsing ---------------------------------- #

def parse_obj(path: Path):
    """Parse a minimal OBJ with v / l records (f is ignored on purpose here)."""
    verts: List[List[float]] = []
    lines: List[Tuple[int, int]] = []

    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            s = raw.strip()
            if not s or s.startswith("#"):
                continue
            if s.startswith("v "):
                _, xs, ys, zs, *rest = s.split()
                verts.append([float(xs), float(ys), float(zs)])
            elif s.startswith("l "):
                parts = s.split()[1:]
                vids = [int(p.split("/")[0]) - 1 for p in parts]
                for a, b in zip(vids, vids[1:]):
                    if a != b:
                        lines.append((min(a, b), max(a, b)))  # undirected, normalized

    V = np.asarray(verts, dtype=np.float64)
    # dedupe lines
    lines = sorted(set(lines))
    return V, lines

# ------------------------ Graph & cycles ----------------------------------- #

def build_adj(nv: int, lines: List[Tuple[int,int]]) -> List[List[int]]:
    adj = [[] for _ in range(nv)]
    for a, b in lines:
        adj[a].append(b)
        adj[b].append(a)
    # sort for determinism
    for lst in adj:
        lst.sort()
    return adj


def find_simple_cycles(adj: List[List[int]], cycle_len_max: int = 32) -> List[List[int]]:
    """Enumerate simple cycles in an undirected graph up to max length.
    Returns cycles as vertex index lists in order (closed implied: first!=last).
    Deduplicates cycles using canonical rotation/min‑lex and direction.
    """
    n = len(adj)
    cycles_set: Set[Tuple[int,...]] = set()
    cycles: List[List[int]] = []

    def canonicalize(path: List[int]) -> Tuple[int,...]:
        # rotate to start from minimal index; pick orientation with smaller tuple
        m = min(path)
        idxs = [i for i, v in enumerate(path) if v == m]
        best = None
        for i in idxs:
            rot = path[i:] + path[:i]
            r1 = tuple(rot)
            r2 = tuple(reversed(rot))
            cand = r1 if r1 < r2 else r2
            if best is None or cand < best:
                best = cand
        return best

    visited = [False]*n

    def dfs(start: int, u: int, parent: int, stack: List[int], blocked: Set[int]):
        if len(stack) > cycle_len_max:
            return
        for v in adj[u]:
            if v == parent:
                continue
            if v == start and len(stack) >= 3:
                cyc = stack.copy()
                key = canonicalize(cyc)
                if key not in cycles_set:
                    cycles_set.add(key)
                    cycles.append(list(key))
                continue
            if v in blocked or v in stack:
                continue
            stack.append(v)
            dfs(start, v, u, stack, blocked)
            stack.pop()

    # iterate vertices in order to keep determinism; block lower ids to avoid dup roots
    blocked_global: Set[int] = set()
    for s in range(n):
        if not adj[s]:
            continue
        dfs(s, s, -1, [s], blocked_global)
        blocked_global.add(s)
    return cycles

In [ ]:
# -------------------- Plane fitting & projection --------------------------- #

def fit_plane(points: np.ndarray) -> Tuple[np.ndarray, np.ndarray, float]:
    """Return (origin, normal, rms) for best‑fit plane via SVD."""
    c = points.mean(0)
    Q = points - c
    # covariance SVD
    U, S, VT = np.linalg.svd(Q, full_matrices=False)
    n = VT[-1, :]
    # RMS distance to plane
    dists = Q @ n
    rms = float(np.sqrt(np.mean(dists**2)))
    return c, n / (np.linalg.norm(n)+1e-12), rms


def plane_basis(n: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # Create orthonormal basis (u,v) in plane, given normal n
    n = n / (np.linalg.norm(n)+1e-12)
    a = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(a, n)) > 0.9:
        a = np.array([0.0, 1.0, 0.0])
    u = np.cross(n, a); u /= (np.linalg.norm(u)+1e-12)
    v = np.cross(n, u); v /= (np.linalg.norm(v)+1e-12)
    return u, v


def project_to_plane(P: np.ndarray, origin: np.ndarray, n: np.ndarray) -> np.ndarray:
    u, v = plane_basis(n)
    Q = P - origin
    return np.stack([Q @ u, Q @ v], axis=1)

# --------------------- 2D polygon utils & triangulation -------------------- #

def polygon_area2d(poly: np.ndarray) -> float:
    x, y = poly[:,0], poly[:,1]
    return 0.5 * float(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1)))


def segments_intersect(a,b,c,d) -> bool:
    def orient(p,q,r):
        return (q[0]-p[0])*(r[1]-p[1]) - (q[1]-p[1])*(r[0]-p[0])
    def onseg(p,q,r):
        return min(p[0],r[0]) - 1e-12 <= q[0] <= max(p[0],r[0]) + 1e-12 and \
               min(p[1],r[1]) - 1e-12 <= q[1] <= max(p[1],r[1]) + 1e-12
    o1 = orient(a,b,c); o2 = orient(a,b,d); o3 = orient(c,d,a); o4 = orient(c,d,b)
    if (o1*o2 < 0) and (o3*o4 < 0):
        return True
    # collinear cases
    if abs(o1) < 1e-12 and onseg(a,c,b): return True
    if abs(o2) < 1e-12 and onseg(a,d,b): return True
    if abs(o3) < 1e-12 and onseg(c,a,d): return True
    if abs(o4) < 1e-12 and onseg(c,b,d): return True
    return False


def simple_polygon(poly: np.ndarray) -> bool:
    m = len(poly)
    for i in range(m):
        a = poly[i]; b = poly[(i+1)%m]
        for j in range(i+1, m):
            if abs(i - j) <= 1 or (i == 0 and j == m-1):
                continue
            c = poly[j]; d = poly[(j+1)%m]
            if segments_intersect(a,b,c,d):
                return False
    return True


def earclip_triangulate(poly: np.ndarray) -> List[Tuple[int,int,int]]:
    m = len(poly)
    idxs = list(range(m))
    tris = []
    if m < 3:
        return tris
    area = polygon_area2d(poly)
    if abs(area) < 1e-18:
        return tris
    ccw = area > 0
    def is_convex(i0,i1,i2):
        a,b,c = poly[i0], poly[i1], poly[i2]
        cross = (b[0]-a[0])*(c[1]-a[1]) - (b[1]-a[1])*(c[0]-a[0])
        return cross > 1e-14 if ccw else cross < -1e-14
    def point_in_tri(p,a,b,c):
        # barycentric in 2D
        v0 = b-a; v1 = c-a; v2 = p-a
        den = v0[0]*v1[1]-v0[1]*v1[0]
        if abs(den) < 1e-20:
            return False
        u = (v2[0]*v1[1]-v2[1]*v1[0])/den
        v = (v0[0]*v2[1]-v0[1]*v2[0])/den
        return u >= -1e-12 and v >= -1e-12 and u+v <= 1+1e-12
    guard = 0
    while len(idxs) > 2 and guard < 10000:
        guard += 1
        ear_found = False
        k = len(idxs)
        for j in range(k):
            i0, i1, i2 = idxs[(j-1)%k], idxs[j], idxs[(j+1)%k]
            if not is_convex(i0,i1,i2):
                continue
            a,b,c = poly[i0], poly[i1], poly[i2]
            # check no other point inside
            ok = True
            for q in idxs:
                if q in (i0,i1,i2):
                    continue
                if point_in_tri(poly[q], a,b,c):
                    ok = False; break
            if ok:
                tris.append((i0,i1,i2))
                del idxs[j]
                ear_found = True
                break
        if not ear_found:
            # likely self-intersecting or nearly collinear; abort
            break
    return tris

In [ ]:
# ------------------- Build faces from wireframe ---------------------------- #

def derive_faces_from_lines(V: np.ndarray, lines: List[Tuple[int,int]], plane_tol: float, cycle_len_max: int):
    adj = build_adj(len(V), lines)
    cycles = find_simple_cycles(adj, cycle_len_max)
    faces: List[Dict] = []
    for cyc in cycles:
        pts = V[np.array(cyc)]
        origin, normal, rms = fit_plane(pts)
        if rms > plane_tol:
            continue
        poly2d = project_to_plane(pts, origin, normal)
        if abs(polygon_area2d(poly2d)) < 1e-10:
            continue
        if not simple_polygon(poly2d):
            continue
        tris = earclip_triangulate(poly2d)
        if not tris:
            continue
        faces.append({
            'vertex_ids': cyc,
            'origin': origin,
            'normal': normal,
            'poly2d': poly2d,
            'tris2d': tris,
        })
    return faces

# ---------------------- Point-in-face distance ----------------------------- #

def point_face_distance(p: np.ndarray, face: Dict) -> Tuple[float, bool]:
    origin = face['origin']; n = face['normal']
    # signed distance to plane
    d = float(np.dot(p - origin, n))
    # project to plane 2D
    u, v = plane_basis(n)
    k2 = np.array([np.dot(p-origin, u), np.dot(p-origin, v)], dtype=np.float64)
    # inside test via triangulation
    inside = False
    for (i,j,k) in face['tris2d']:
        a = face['poly2d'][i]; b = face['poly2d'][j]; c = face['poly2d'][k]
        # barycentric in 2D
        v0 = b-a; v1 = c-a; v2 = k2-a
        den = v0[0]*v1[1]-v0[1]*v1[0]
        if abs(den) < 1e-20:
            continue
        u1 = (v2[0]*v1[1]-v2[1]*v1[0])/den
        v1b = (v0[0]*v2[1]-v0[1]*v2[0])/den
        if u1 >= -1e-12 and v1b >= -1e-12 and u1+v1b <= 1+1e-12:
            inside = True
            break
    return abs(d), inside

# --------------------------- IO utilities ---------------------------------- #

def stream_xyz(path: Path, chunk_size: int) -> Iterable[np.ndarray]:
    buf: List[List[float]] = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            parts = s.split()
            try:
                row = [float(x) for x in parts]
            except ValueError:
                continue
            buf.append(row)
            if len(buf) >= chunk_size:
                yield np.asarray(buf, dtype=np.float64)
                buf.clear()
    if buf:
        yield np.asarray(buf, dtype=np.float64)


def write_xyz_with_face(in_chunk: np.ndarray, face_ids: np.ndarray, out_fh):
    assert len(in_chunk) == len(face_ids)
    for row, fid in zip(in_chunk, face_ids):
        vals = list(row) + [int(fid)]
        out_fh.write(" ".join(str(v) for v in vals) + "\n")

In [ ]:
# ------------------------------- Main -------------------------------------- #

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--xyz", required=True, type=Path)
    ap.add_argument("--obj", required=True, type=Path)
    ap.add_argument("--out", required=True, type=Path)
    ap.add_argument("--max_distance", type=float, default=1.0,
                    help="Max perpendicular distance to accept a face (units of CRS). 0 disables.")
    ap.add_argument("--plane_tol", type=float, default=0.02,
                    help="Max RMS off‑plane error to accept a cycle as a face (units of CRS).")
    ap.add_argument("--cycle_len_max", type=int, default=32,
                    help="Maximum cycle length to consider when deriving faces from lines.")
    ap.add_argument("--chunk_size", type=int, default=200000)
    args = ap.parse_args()

    V, lines = parse_obj(args.obj)
    if len(V) == 0:
        print("ERROR: OBJ contains no vertices.", file=sys.stderr)
        sys.exit(2)
    if len(lines) == 0:
        print("ERROR: OBJ contains no line segments (l).", file=sys.stderr)
        sys.exit(3)

    # Build faces from wireframe lines
    faces = derive_faces_from_lines(V, lines, args.plane_tol, args.cycle_len_max)
    if not faces:
        print("ERROR: No valid faces could be derived from the wireframe.", file=sys.stderr)
        sys.exit(4)

    # KD‑tree on face centroids for candidate filtering
    cents = []
    for f in faces:
        c3 = V[np.array(f['vertex_ids'])].mean(0)
        cents.append(c3)
    centroids = np.vstack(cents)
    kdt = cKDTree(centroids)

    with args.out.open("w", encoding="utf-8") as out_fh:
        for chunk in stream_xyz(args.xyz, args.chunk_size):
            if chunk.shape[1] < 3:
                continue
            P = chunk[:, :3]
            out_ids = np.full((len(P),), -1, dtype=np.int64)
            # Candidate search per point
            k = min(48, len(faces))
            dists, idxs = kdt.query(P, k=k, workers=-1)
            if k == 1:
                idxs = idxs.reshape(-1,1)
            for i, p in enumerate(P):
                best = (math.inf, -1)
                for fid in idxs[i]:
                    dist, inside = point_face_distance(p, faces[int(fid)])
                    if inside and (args.max_distance <= 0 or dist <= args.max_distance):
                        if dist < best[0]:
                            best = (dist, int(fid))
                if best[1] != -1:
                    out_ids[i] = best[1]
            write_xyz_with_face(chunk, out_ids, out_fh)

    print(f"Derived {len(faces)} faces | Wrote: {args.out}")

In [ ]:
xyz_file = Path("/path/to/your/input.xyz")
obj_file = Path("/path/to/your/wireframe.obj")
out_file = Path("/path/to/your/output.xyz")

MAX_DISTANCE = 1.0
PLANE_TOL = 0.02
CYCLE_LEN_MAX = 32
CHUNK_SIZE = 200000

def main():
    # Use the module-level paths / params instead of argparse
    V, lines = parse_obj(obj_file)
    if len(V) == 0:
        print("ERROR: OBJ contains no vertices.", file=sys.stderr)
        return
    if len(lines) == 0:
        print("ERROR: OBJ contains no line segments (l).", file=sys.stderr)
        return

    faces = derive_faces_from_lines(V, lines, PLANE_TOL, CYCLE_LEN_MAX)
    if not faces:
        print("ERROR: No valid faces could be derived from the wireframe.", file=sys.stderr)
        return

    cents = []
    for f in faces:
        c3 = V[np.array(f['vertex_ids'])].mean(0)
        cents.append(c3)
    centroids = np.vstack(cents)
    kdt = cKDTree(centroids)

    with out_file.open("w", encoding="utf-8") as out_fh:
        for chunk in stream_xyz(xyz_file, CHUNK_SIZE):
            if chunk.shape[1] < 3:
                continue
            P = chunk[:, :3]
            out_ids = np.full((len(P),), -1, dtype=np.int64)
            k = min(48, len(faces))
            dists, idxs = kdt.query(P, k=k, workers=-1)
            if k == 1:
                idxs = idxs.reshape(-1, 1)
            for i, p in enumerate(P):
                best = (math.inf, -1)
                for fid in idxs[i]:
                    dist, inside = point_face_distance(p, faces[int(fid)])
                    if inside and (MAX_DISTANCE <= 0 or dist <= MAX_DISTANCE):
                        if dist < best[0]:
                            best = (dist, int(fid))
                if best[1] != -1:
                    out_ids[i] = best[1]
            write_xyz_with_face(chunk, out_ids, out_fh)

    print(f"Derived {len(faces)} faces | Wrote: {out_file}")